In [4]:
import pandas as pd
from tqdm import tqdm
import re

In [5]:
sequence_df1 = pd.read_csv("../train_data/protein_sequences_output_sample_refine_col.csv")
sequence_df2 = pd.read_csv("../train_data/protein_sequences_output_sample_refine_col_uniprot.csv")
train = pd.read_csv('../train_data/train.csv')

In [6]:
train = train.fillna("WT")

In [27]:
train.head()

,ID,SUBCLASS,BRCA1,HMGB3,PLXNB2,NOTCH2,RYR2,ALB,BTG1,SMC1A,...,PEX6,B2M,MYLK,BTG2,CACNA1B,PABPC1,FGFR3,CLIP2,CDH1,HELZ2
0,TCGA-OR-A5J1-01A-11D-A29I-10,ACC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TCGA-OR-A5J4-01A-11D-A29I-10,ACC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,T1034N,WT,WT,WT,WT,WT,WT,WT
2,TCGA-OR-A5J5-01A-11D-A29I-10,ACC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TCGA-OR-A5J6-01A-31D-A29I-10,ACC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TCGA-OR-A5J7-01A-11D-A29I-10,ACC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [7]:
pattern = r"^[A-Za-z]\d+[A-Za-z]$"

In [8]:
# 추출 함수 정의
def extract_alpha_numeric(value):
    match = re.match(r"([A-Za-z])(\d+)", value)
    if match:
        return match.group(1), match.group(2)
    return None, None

In [ ]:
percent = []
over_len = []
for col in tqdm(sequence_df.columns):
    num = 0
    df = train[train[col] != 'WT'][col]
    filtered_df_strict = df[df.str.match(pattern)]
    extracted_values = filtered_df_strict.apply(lambda x: pd.Series(extract_alpha_numeric(x), index=["Letter", "Number"]))
    ch = []
    for i in range(len(extracted_values)):
        try:
            ch.append(sequence_df[col][0][int(extracted_values.iloc[i,1])-1]==extracted_values.iloc[i,0])
        except Exception as e:
            num += 1
            print(f"An error occurred: {e}")
            ch.append(False)
    
    true_count = sum(ch)
    total_count = len(ch)
    if total_count == 0:
        total_count = "NO COUNT"
    else:
        true_ratio = true_count / total_count
    over_len.append((col,num,total_count))
    # print(col, total_count)
    percent.append((col,true_ratio))

In [32]:
percent, over_len

([('BRCA1', 0.1206896551724138),
  ('HMGB3', 1.0),
  ('PLXNB2', 0.025),
  ('NOTCH2', 0.5074626865671642),
  ('RYR2', 1.0),
  ('ALB', 1.0),
  ('BTG1', 1.0),
  ('SMC1A', 0.01818181818181818),
  ('AQP7', 0.7777777777777778),
  ('PIM1', 0.07692307692307693),
  ('FBXW7', 1.0),
  ('CDKN1A', 0.1875),
  ('ENPP2', 0.0),
  ('AHNAK', 0.009708737864077669),
  ('COL6A3', 0.04591836734693878),
  ('PLEC', 0.04195804195804196),
  ('SOX9', 1.0),
  ('CLTC', 0.1276595744680851),
  ('KIF3B', 1.0),
  ('PKHD1', 0.8057851239669421),
  ('CREBBP', 0.4175824175824176),
  ('VHL', 0.3114754098360656),
  ('RNF213', 0.6434782608695652),
  ('AMOT', 1.0),
  ('SPTB', 0.6732673267326733),
  ('TGFBR2', 0.10638297872340426),
  ('RYR1', 0.673992673992674),
  ('SMC3', 1.0),
  ('XDH', 0.7910447761194029),
  ('ALMS1', 1.0),
  ('AKT1', 1.0),
  ('MYH1', 1.0),
  ('DLC1', 0.06097560975609756),
  ('SPOP', 1.0),
  ('CELSR1', 0.038461538461538464),
  ('SPTAN1', 0.04838709677419355),
  ('LRP1', 0.0),
  ('YWHAQ', 1.0),
  ('MYH2', 1.0

# 두개 비교

In [9]:
percent = []
over_len = []
dic = {}
source_dict = {}  # 염기서열 출처를 저장할 딕셔너리

for col in tqdm(sequence_df1.columns):
    num = 0
    df = train[train[col] != 'WT'][col]
    filtered_df_strict = df[df.str.match(pattern)]
    extracted_values = filtered_df_strict.apply(lambda x: pd.Series(extract_alpha_numeric(x), index=["Letter", "Number"]))
    
    ch1 = []
    for i in range(len(extracted_values)):
        try:
            ch1.append(sequence_df1[col][0][int(extracted_values.iloc[i, 1]) - 1] == extracted_values.iloc[i, 0])
        except Exception as e:
            num += 1
            # print(f"An error occurred: {e}")
            ch1.append(False)
    
    true_count1 = sum(ch1)
    total_count1 = len(ch1)
    true_ratio1 = true_count1 / total_count1 if total_count1 != 0 else 0

    ch2 = []
    for i in range(len(extracted_values)):
        try:
            ch2.append(sequence_df2[col][0][int(extracted_values.iloc[i, 1]) - 1] == extracted_values.iloc[i, 0])
        except Exception as e:
            num += 1
            print(f"An error occurred: {e}")
            ch2.append(False)
    
    true_count2 = sum(ch2)
    total_count2 = len(ch2)
    true_ratio2 = true_count2 / total_count2 if total_count2 != 0 else 0

    # Compare true_ratio1 and true_ratio2, add appropriate sequence to dic and source_dict
    if true_ratio1 >= true_ratio2:
        dic[col] = sequence_df1[col][0]
        source_dict[col] = "NCBI"
    else:
        dic[col] = sequence_df2[col][0]
        source_dict[col] = "Uniprot"
    
    over_len.append((col, num, len(extracted_values)))
    percent.append((col, true_ratio1 if true_ratio1 >= true_ratio2 else true_ratio2))

# Output the final results
print("Dictionary with selected sequences:", dic)
print("Source dictionary:", source_dict)
print("Over length details:", over_len)
print("Percentages of matches:", percent)


  0%|          | 0/148 [00:00<?, ?it/s]

 17%|█▋        | 25/148 [00:02<00:14,  8.62it/s]

An error occurred: string index out of range
An error occurred: string index out of range
An error occurred: string index out of range
An error occurred: string index out of range
An error occurred: string index out of range
An error occurred: string index out of range


 42%|████▏     | 62/148 [00:08<00:08, 10.49it/s]

An error occurred: string index out of range


100%|██████████| 148/148 [00:19<00:00,  7.71it/s]

Dictionary with selected sequences: {'BRCA1': 'MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKFCMLKLLNQKKGPSQCPLCKNDITKRSLQESTRFSQLVEELLKIICAFQLDTGLEYANSYNFAKKENNSPEHLKDEVSIIQSMGYRNRAKRLLQSEPENPSLQETSLSVQLSNLGTVRTLRTKQRIQPQKTSVYIELGSDSSEDTVNKATYCSVGDQELLQITPQGTRDEISLDSAKKAACEFSETDVTNTEHHQPSNNDLNTTEKRAAERHPEKYQGSSVSNLHVEPCGTNTHASSLQHENSSLLLTKDRMNVEKAEFCNKSKQPGLARSQHNRWAGSKETCNDRRTPSTEKKVDLNADPLCERKEWNKQKLPCSENPRDTEDVPWITLNSSIQKVNEWFSRSDELLGSDDSHDGESESNAKVADVLDVLNEVDEYSGSSEKIDLLASDPHEALICKSERVHSKSVESNIEDKIFGKTYRKKASLPNLSHVTENLIIGAFVTEPQIIQERPLTNKLKRKRRPTSGLHPEDFIKKADLAVQKTPEMINQGTNQTEQNGQVMNITNSGHENKTKGDSIQNEKNPNPIESLEKESAFKTKAEPISSSISNMELELNIHNSKAPKKNRLRRKSSTRHIHALELVVSRNLSPPNCTELQIDSCSSSEEIKKKKYNQMPVRHSRNLQLMEGKEPATGAKKSNKPNEQTSKRHDSDTFPELKLTNAPGSFTKCSNTSELKEFVNPSLPREEKEEKLETVKVSNNAEDPKDLMLSGERVLQTERSVESSSISLVPGTDYGTQESISLLEVSTLGKAKTEPNKCVSQCAAFENPKGLIHGCSKDNRNDTEGFKYPLGHEVNHSRETSIEMEESELDAQYLQNTFKVSKRQSFAPFSNPGNAEEECATFSAHSGSLKKQSPKVTFECEQKEENQGKNESNIKPVQTVNITAGFPVVGQKDKPVDNAKCSIKGGSRFC

In [10]:
print("Source dictionary:", source_dict)
print("Over length details:", over_len)
print("Percentages of matches:", percent)

Source dictionary: {'BRCA1': 'Uniprot', 'HMGB3': 'NCBI', 'PLXNB2': 'Uniprot', 'NOTCH2': 'Uniprot', 'RYR2': 'NCBI', 'ALB': 'NCBI', 'BTG1': 'NCBI', 'SMC1A': 'Uniprot', 'AQP7': 'NCBI', 'PIM1': 'Uniprot', 'FBXW7': 'NCBI', 'CDKN1A': 'Uniprot', 'ENPP2': 'Uniprot', 'AHNAK': 'Uniprot', 'COL6A3': 'Uniprot', 'PLEC': 'Uniprot', 'SOX9': 'NCBI', 'CLTC': 'Uniprot', 'KIF3B': 'NCBI', 'PKHD1': 'Uniprot', 'CREBBP': 'Uniprot', 'VHL': 'Uniprot', 'RNF213': 'Uniprot', 'AMOT': 'NCBI', 'SPTB': 'Uniprot', 'TGFBR2': 'Uniprot', 'RYR1': 'Uniprot', 'SMC3': 'NCBI', 'XDH': 'Uniprot', 'ALMS1': 'NCBI', 'AKT1': 'NCBI', 'MYH1': 'NCBI', 'DLC1': 'NCBI', 'SPOP': 'NCBI', 'CELSR1': 'Uniprot', 'SPTAN1': 'Uniprot', 'LRP1': 'Uniprot', 'YWHAQ': 'NCBI', 'MYH2': 'NCBI', 'ATRX': 'Uniprot', 'COL5A1': 'Uniprot', 'PIK3CA': 'NCBI', 'PCLO': 'Uniprot', 'SPTA1': 'Uniprot', 'NF1': 'Uniprot', 'FBN2': 'Uniprot', 'RELN': 'NCBI', 'MXRA5': 'NCBI', 'NOTCH1': 'NCBI', 'KMT2D': 'NCBI', 'NLRC5': 'Uniprot', 'TSC2': 'Uniprot', 'PEG3': 'Uniprot', 'VWF'

In [11]:
uniprot_count = sum(1 for source in source_dict.values() if source == "Uniprot")
print(f"Number of Uniprot values: {uniprot_count}")


Number of Uniprot values: 99


In [12]:
# Count the number of columns with a true_ratio below 0.8
low_percent_count = sum(1 for col, true_ratio in percent if true_ratio < 0.8)
print(f"Number of columns with percent below 0.8: {low_percent_count}")

Number of columns with percent below 0.8: 10


In [13]:
low_percent_columns = [(col, true_ratio) for col, true_ratio in percent if true_ratio < 0.8]
low_percent_columns

[('ALB', 0.2777777777777778),
 ('AQP7', 0.4358974358974359),
 ('ALMS1', 0.20967741935483872),
 ('DLC1', 0.028368794326241134),
 ('TAP1', 0.15625),
 ('GNAS', 0.19444444444444445),
 ('PRKDC', 0.3609467455621302),
 ('DST', 0.5934065934065934),
 ('MET', 0.6545454545454545),
 ('HELZ2', 0.75)]

In [14]:
df = pd.DataFrame([dic])

In [16]:
df.head()

,BRCA1,HMGB3,PLXNB2,NOTCH2,RYR2,ALB,BTG1,SMC1A,AQP7,PIM1,...,PEX6,B2M,MYLK,BTG2,CACNA1B,PABPC1,FGFR3,CLIP2,CDH1,HELZ2
0,MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKF...,MAKGDPKKPKGKMSAYAFFVQTCREEHKKKNPEVPVNFAEFSKKCS...,MALQLWALTLLGLLGAGASLRPRKLDFFRSEKELNHLAVDEASGVV...,MPALRPALLWALLALWLCCAAPAHALQCRDGYEPCVNEGMCVTYHN...,MADGGEGEDEIQFLRTDDEVVLQCTATIHKEQQKLCLAAEGFGNRL...,MKWVTFISLLFLFSSAYSRGVFRRDAHKSEVAHRFKDLGEENFKAL...,MHPFYTRAATMIGEIAAAVSFISKFLRTKGLTSERQLQTFSQSLQE...,MGFLKLIEIENFKSYKGRQIIGPFQRFTAIIGPNGSGKSNLMDAIS...,MVQASGHRRSTRGSKMVSWSVIAKIQEILQRKMVREFLAEFMSTYV...,MLLSKINSLAHLRAAPCNDLHATKLAPGKEKEPLESQYQVGPLLGS...,...,MALAVLRVLEPFPTETPPLAVLLPPGGPWPAAELGLVLALRPAGES...,MSRSVALAVLALLSLSGLEAIQRTPKIQVYSRHPAENGKSNFLNCY...,MGDVKLVASSHISKTSLSVDPSRVDSMPLTEAPAFILPPRNLCIKE...,MSHGKGTDMLPEIAAAVGFLSSLLRTRGCVSEQRLKVFSGALQEAL...,MVRFGDELGGRYGGPGGGERARGGGAGGAGGPGPGGLQPGQRVLYK...,MNPSAPSYPMASLYVGDLHPDVTEAMLYEKFSPAGPILSIRVCRDM...,MGAPACALALCVAVAIVAGASSESLGTEQRVVGRAAEVPGPEPGQQ...,MQKPSGLKPPGRGGKHSSPMGRTSTGSASSSAAVAASSKEGSPLHK...,MGPWSRSLSALLLLLQVSSWLCQEPEPCHPGFDAESYTFTVPRRHL...,MAVWEAEQLGGLQRGDLLTPPAPDGDGRTAPLGQPPGAQLYCPACL...


In [17]:
df.to_csv("../train_data/protein_sequences_output.csv")

In [18]:
df.to_pickle('../train_data/protein_sequences_output.pkl')


In [23]:
high_percent_columns = [col for col, true_ratio in percent if true_ratio == 1.0]


In [24]:
high_percent_columns

['BRCA1',
 'HMGB3',
 'PLXNB2',
 'NOTCH2',
 'RYR2',
 'BTG1',
 'SMC1A',
 'PIM1',
 'FBXW7',
 'CDKN1A',
 'ENPP2',
 'COL6A3',
 'PLEC',
 'SOX9',
 'CLTC',
 'KIF3B',
 'PKHD1',
 'CREBBP',
 'VHL',
 'RNF213',
 'AMOT',
 'TGFBR2',
 'RYR1',
 'SMC3',
 'XDH',
 'AKT1',
 'MYH1',
 'SPOP',
 'CELSR1',
 'SPTAN1',
 'LRP1',
 'YWHAQ',
 'MYH2',
 'ATRX',
 'COL5A1',
 'PIK3CA',
 'SPTA1',
 'NF1',
 'FBN2',
 'RELN',
 'MXRA5',
 'NOTCH1',
 'KMT2D',
 'NLRC5',
 'TSC2',
 'PEG3',
 'VWF',
 'SYMPK',
 'SOWAHC',
 'LAMA1',
 'CASP8',
 'ARID5B',
 'IGSF1',
 'CMPK2',
 'EGFR',
 'NLRP3',
 'IDH2',
 'MAPT',
 'PKD1',
 'TNFAIP3',
 'SCN10A',
 'COL4A1',
 'ZBTB10',
 'TP53',
 'HUWE1',
 'KIT',
 'AXIN1',
 'PTEN',
 'ITIH1',
 'IRF4',
 'RB1',
 'MTOR',
 'FASN',
 'CEBPA',
 'STAT3',
 'BMP2K',
 'LRIG1',
 'EIF4G3',
 'HRAS',
 'GOLGA4',
 'CTCF',
 'COL11A1',
 'TOP2A',
 'CHST2',
 'CHEK2',
 'NCOA6',
 'TM7SF2',
 'COL5A3',
 'NFKB2',
 'MAP3K1',
 'TG',
 'ERCC2',
 'GATA3',
 'CDH8',
 'APC',
 'CTNNB1',
 'ELF3',
 'LAMA2',
 'PTPRD',
 'RET',
 'CDC27',
 'ARHGAP5',
 '